# Transaction

In [ ]:
import redis
from threading import Thread

rd = redis.Redis(host='localhost', port=6379, db=0, decode_responses=True)

## 1. Pipeline

### 1.1. Batch commands

In [ ]:
try:
    with rd.pipeline(transaction=False) as pipe:
        r = pipe.set(name='a', value=100) \
            .incr(name='a') \
            .incr(name='a', amount=10) \
            .execute()
        print('* the result of batch execute is: {}'.format(r))

    r = rd.get(name='a')
    print('* the result of batch commands is: {}'.format(r))
finally:
    rd.delete('a')

### 1.2. Transaction operate

#### 1.2.1. Pipeline with transaction

- `MULTI`: start transaction
- `EXEC`: execute batch commands
- `DISCARD`: discard all batch command in a transaction

In [ ]:
try:
    with rd.pipeline() as pipe:
        try:
            pipe.multi()  # start transaction
            
            r = pipe.set(name='a', value=100) \
                .incr(name='a') \
                .incr(name='a', amount=10) \
                .execute()  # run batch command

            print('* the result of batch command is: {}'.format(r))
        except Exception as err:
            print('* error caused is: {}'.format(err))

    # validate batch command
    r = rd.get(name='a')
    print('* the result of batch command is: {}'.format(r))
finally:
    rd.delete('a')

#### 1.2.2. Transaction with watch

- `WATCH key [key …]`: watch the specific keys
- `UNWATCH`: unwatch all keys

##### 1.2.2.1. Change key value in other process

In [ ]:
def set_a_in_other_process(name=None, value=None):
    def runner():
        rd = redis.Redis(host='localhost', port=6379, db=0, decode_responses=True)
        try:
            if name is not None:
                rd.set(name=name, value=value)
        finally:
            rd.connection_pool.disconnect()

    t = Thread(target=runner)
    t.start()
    t.join()

##### 1.2.2.2. Watch the kay but no change

In [ ]:
try:
    with rd.pipeline() as pipe:
        try:
            pipe.watch('a')
            pipe.multi()
            pipe.set('a', 100)

            set_a_in_other_process(name=None, value=None)

            r = pipe.execute()
            print('* the result of batch command is: {}'.format(r))
        except Exception as err:
            print('* error caused is: {}'.format(err))

    r = rd.get('a')
    print('* the result of batch command is: {}'.format(r))
finally:
    rd.delete('a')

##### 1.2.2.3. Watch the kay and key changed in other process

In [ ]:
try:
    with rd.pipeline() as pipe:
        try:
            pipe.watch('a')
            pipe.multi()
            pipe.set('a', 100)

            set_a_in_other_process(name='a', value=100)

            r = pipe.execute()
            print('* the result of batch command is: {}'.format(r))
        except Exception as err:
            print('* error caused is: {}'.format(err))

    r = rd.get('a')
    print('* the result of batch command is: {}'.format(r))
finally:
    rd.delete('a')

##### 1.2.2.4. Watch and unwatch

In [ ]:
try:
    with rd.pipeline() as pipe:
        try:
            pipe.watch('a')
            pipe.unwatch()
            
            pipe.multi()
            pipe.set('a', 100)

            set_a_in_other_process(name='a', value=200)

            r = pipe.execute()
            print('* the result of batch command is: {}'.format(r))
        except Exception as err:
            print('* error caused is: {}'.format(err))

    r = rd.get('a')
    print('* the result of batch command is: {}'.format(r))
finally:
    rd.delete('a')